In [1]:
import numpy as np
import pandas as pd
from textblob import TextBlob
import glob
import re
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

In [2]:
train=pd.read_pickle('./dataset2Cleaned.pkl')

test=pd.read_pickle('./dataset1Cleaned.pkl')

In [3]:

def text_processing(tweet):
    
    tweet= tweet.lower()
    
    #Removing hyperlinks from the tweet
    tweet_no_links=re.sub(r'http\S+', '', tweet)
    
    #Generating the list of words in the tweet (hashtags and other punctuations removed)
    def form_sentence(tweet):
        tweet_blob = TextBlob(tweet)
        return ' '.join(tweet_blob.words)
    new_tweet = form_sentence(tweet_no_links)
    
    #Removing stopwords and words with unusual symbols
    def no_user_alpha(tweet):
        tweet_list = [ele for ele in tweet.split() if ele != 'user']
        clean_tokens = [t for t in tweet_list if re.match(r'[^\W\d]*$', t)]
        clean_s = ' '.join(clean_tokens)
        clean_mess = [word for word in clean_s.split() if word not in stopwords.words('english')]
        return clean_mess
    no_punc_tweet = no_user_alpha(new_tweet)
    
    #Normalizing the words in tweets 
    def normalization(tweet_list):
        lem = WordNetLemmatizer()
        normalized_tweet = []
        for word in tweet_list:
            normalized_text = lem.lemmatize(word,'v')
            normalized_tweet.append(normalized_text)
        return " ".join(normalized_tweet)
    
    
    return normalization(no_punc_tweet)

In [4]:
train['text']=train['text'].apply(text_processing)
test['text']=test['text'].apply(text_processing)


In [5]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [6]:
texts=train.text
tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\'',
                      lower=True)
tokenizer.fit_on_texts(texts)
sequences_train = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 18063 unique tokens.


In [7]:
X_train = pad_sequences(sequences_train)
y_train = np.asarray(train.Informativeness)
print('Shape of X train :', X_train.shape)
print('Shape of label train :', y_train.shape)

Shape of X train : (15745, 19)
Shape of label train : (15745,)


In [8]:
max(train.text.apply(lambda x: len(x.split(' '))))

18

In [9]:
train.shape

(15745, 3)

In [10]:
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

from gensim.models.keyedvectors import KeyedVectors

word_vectors = KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True)

EMBEDDING_DIM=300
vocabulary_size=len(word_index)+1
embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))
for word, i in word_index.items():
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)

del(word_vectors)

from keras.layers import Embedding
embedding_layer = Embedding(vocabulary_size,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            trainable=True)

In [11]:
embedding_matrix.shape

(18064, 300)

In [39]:
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPooling2D, Dropout,concatenate
from keras.layers.core import Reshape, Flatten
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.models import Model
from keras import regularizers
sequence_length = X_train.shape[1]

filter_sizes = [3,4,5]
num_filters = 150
drop = 0.5



inputs = Input(shape=(sequence_length,))
embedding = embedding_layer(inputs)
reshape = Reshape((sequence_length,EMBEDDING_DIM,1))(embedding)


conv_0 = Conv2D(num_filters, (filter_sizes[0], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_1 = Conv2D(num_filters, (filter_sizes[1], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_2 = Conv2D(num_filters, (filter_sizes[2], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)

maxpool_0 = MaxPooling2D((sequence_length - filter_sizes[0] + 1, 1), strides=(1,1))(conv_0)
maxpool_1 = MaxPooling2D((sequence_length - filter_sizes[1] + 1, 1), strides=(1,1))(conv_1)
maxpool_2 = MaxPooling2D((sequence_length - filter_sizes[2] + 1, 1), strides=(1,1))(conv_2)

merged_tensor = concatenate([maxpool_0, maxpool_1, maxpool_2], axis=1)
flatten = Flatten()(merged_tensor)
reshape = Reshape((3*num_filters,))(flatten)
dropout = Dropout(drop)(flatten)
output = Dense(units=2, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout)


model = Model(inputs, output)

In [35]:
y_train_extra = np.abs(y_train-1)

In [38]:
np.append(y_train, y_train_extra, axis=1)

(15745, 2)

In [51]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['AUC'])
callbacks = [EarlyStopping(monitor='loss')]
model.fit(X_train, np.append(y_train, y_train_extra, axis=1), batch_size=1000, epochs=10, verbose=1,
         callbacks=callbacks)  


Epoch 1/10
16/16 [==============================] - 20s 1s/step - loss: 0.2857 - auc: 0.9859
Epoch 2/10
16/16 [==============================] - 15s 956ms/step - loss: 0.2624 - auc: 0.9902
Epoch 3/10
16/16 [==============================] - 22s 1s/step - loss: 0.2453 - auc: 0.9921
Epoch 4/10
16/16 [==============================] - 24s 2s/step - loss: 0.2338 - auc: 0.9936
Epoch 5/10
16/16 [==============================] - 18s 1s/step - loss: 0.2267 - auc: 0.9940
Epoch 6/10
16/16 [==============================] - 24s 2s/step - loss: 0.2179 - auc: 0.9950
Epoch 7/10
16/16 [==============================] - 20s 1s/step - loss: 0.2102 - auc: 0.9958
Epoch 8/10
16/16 [==============================] - 20s 1s/step - loss: 0.2050 - auc: 0.9961
Epoch 9/10
16/16 [==============================] - 23s 1s/step - loss: 0.1989 - auc: 0.9966
Epoch 10/10
16/16 [==============================] - 26s 2s/step - loss: 0.1914 - auc: 0.9972


In [52]:
sequences_test=tokenizer.texts_to_sequences(test.text)
X_test = pad_sequences(sequences_test,maxlen=X_train.shape[1])
y_pred=model.predict(X_test)

In [53]:
y_pred

array([[0.9522171 , 0.04778284],
       [0.10866933, 0.89133066],
       [0.9454676 , 0.05453239],
       ...,
       [0.2715427 , 0.72845733],
       [0.15764073, 0.8423593 ],
       [0.76780456, 0.23219544]], dtype=float32)

In [54]:
y_pred_bool =(model.predict(X_test) > 0.5).astype("int32")

In [48]:
y_pred_bool[:,0]

array([1, 0, 1, ..., 0, 0, 1], dtype=int32)

In [ ]:
X_test

In [62]:
y_true = np.asarray(test.Informativeness).reshape(len(test),1)

In [63]:
roc_auc_score(y_true, y_pred_bool[:,0])

0.6295299163640496

In [64]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
print("Acc: {}, f1: {}, roc: {}".format(accuracy_score(y_true, y_pred_bool[:,0]),f1_score(y_true, y_pred_bool[:,0]),roc_auc_score(y_true, y_pred_bool[:,0])))


Acc: 0.6912274905005782, f1: 0.778703082448593, roc: 0.6295299163640496
